# Command Line Interface

> Convert spectra to wavelet images.


In [ ]:
#| default_exp cli

In [ ]:
#| export
from fastcore.script import *
import numpy as np
from uhina.wavelets import (
    convert_to_wavelet_images, 
    create_image_target_csv, 
    estimate_percentiles)
from sklearn.pipeline import Pipeline
from uhina.preprocessing import SNV, TakeDerivative
from uhina.loading import OSSLLoader
from pathlib import Path

In [ ]:
#| exports
@call_parse
def main(
    src: str, # Source file
    dir_out: str, # Output root directory
    img_dir: str='img', # Image directory
    dataset: str='ossl', # Dataset to use
    spectra_type:str='mir', # Spectra type to use
    analytes: str='k.ext_usda.a725_cmolc.kg', # Analytes to use
    n_samples: int=None, # Number of samples to use
    batch_size: int=10, # Batch size to use
    # n_percentile_estimate: int=1000, # Number of samples to use to estimate percentiles
    ):
    "Convert spectra to wavelet images."
    
    if dataset == 'ossl':
        loader = OSSLLoader()
        analytes = 'k.ext_usda.a725_cmolc.kg'
        X, y, wavenumbers, smp_idx, ds_name, ds_label = loader.load_data(analytes, 
                                                                         spectra_type='mir')
    else:
        raise ValueError(f"Dataset {dataset} not supported yet ...")
    
    # Preprocessing pipeline
    pipe = Pipeline([
        ('SNV', SNV()),
        ('Derivative', TakeDerivative())
    ])
    X_trans = pipe.fit_transform(X)
    
    print(f'Creating image target csv in {dir_out} ...')
    create_image_target_csv(smp_idx, y, n_samples=n_samples, output_dir=dir_out)  
    
    print(f'Creating wavelet images in {dir_out} ...')
    convert_to_wavelet_images(X_trans, smp_idx, wavenumbers, 
                              output_dir=Path(dir_out)/img_dir, 
                            #   plot_kwargs={'levels': percentiles_result},
                              n_samples=n_samples, batch_size=batch_size)